## BERT

In [1]:
!pip install transformers datasets tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [30]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import math
import os
import tqdm
from pathlib import Path
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

In [3]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq /content/cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip

--2024-07-22 15:02:55--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  25.3MB/s    in 0.4s    

2024-07-22 15:02:55 (25.3 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]



In [4]:
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

# Preprocess

In [17]:
with open("/content/datasets/movie_conversations.txt", "r", encoding = "iso-8859-1") as file:
  conversations = file.readlines()

with open("/content/datasets/movie_lines.txt", "r", encoding = "iso-8859-1") as file:
  lines = file.readlines()

In [18]:
"""
    Description:
    - Each line in lines has format: "L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n"
        -> Each line is the line number and its text line
    - Each conversation has format: "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']\n"
        -> Each conversation has format a conversation in a list by line number
"""
tag_to_text = {}
conversations_split = []
qa_pairs = []

for line in lines:
  line = line.split("+++$+++")
  tag_to_text[line[0].strip()] = ' '.join(line[-1].replace("\n", "").strip().split())

for conversation in conversations:
  conversation = conversation.split("+++$+++")[-1].replace("\n", "").strip()
  line_tags = conversation.strip("[]").split(",")
  conv_split = []

  for id, tag in enumerate(line_tags):
    # print(tag.strip().strip("'"))
    text = tag_to_text[tag.strip().strip("'")]
    conv_split.append(text)

    if id < len(line_tags) - 1:
      pair = []
      next_text = tag_to_text[line_tags[id + 1].strip().strip("'")]
      pair.append(' '.join(text.split()))
      pair.append(' '.join(next_text.split()))

      qa_pairs.append(pair)

  conversations_split.append(conv_split)

lines = [pair[0] for pair in qa_pairs]

## WordPiece Tokenization

In [20]:
!rm -rf /content/datasets/data

In [21]:
%cd /content/datasets
!mkdir data
%cd data

text_file = 0

for id, line in tqdm.tqdm(enumerate(lines), total = len(lines)):
  if id % 10000 == 0:
    batch_lines = [line]
    text_file += 1
  else:
    batch_lines.append(line)
    if (id + 1) % 10000 == 0:
      with open(f"text_{text_file}.txt", "w", encoding = "utf-8") as file:
        file.write("\n".join(batch_lines))

/content/datasets
/content/datasets/data


100%|██████████| 221616/221616 [00:00<00:00, 1114070.68it/s]


# Training Tokenizer

In [31]:
data_path = "/content/datasets/data"
paths = []
for file in os.listdir(data_path):
  paths.append(os.path.join(data_path, file))

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=5,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

%cd /content/datasets
os.mkdir('./tokenizer')
tokenizer.save_model('./tokenizer', 'bert')

/content/datasets


['./tokenizer/bert-vocab.txt']